In [1]:
from openpyxl import Workbook
import openpyxl
book = openpyxl.load_workbook('Insights.xlsx')
sheet = book.active

# check we're working on the right thing
print(sheet.cell(row=9,column=13).value)

#extract the column data into a list, one item per row, ignoring empty cells
learn_more_about=[]
for row in range(11,862):
    val=sheet.cell(row=row,column=13).value
    if (val is not None): learn_more_about.append(val)
        
#check we've got what we expected
learn_more_about[:10]

#do the conversion to tf-idf, ignoring English stop words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
vectorizer = TfidfVectorizer(max_df=0.5, max_features=500,
                                 min_df=2, stop_words='english',
                                 use_idf=1)
X = vectorizer.fit_transform(learn_more_about)

#run K-means++ with mini-batch
from sklearn.cluster import MiniBatchKMeans
from time import time

km = MiniBatchKMeans(n_clusters=20, init='k-means++', n_init=3, verbose=True)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

#show top 10 words in each cluster
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(20):
    print("Cluster %d:" % i, end='')
    for j in order_centroids[i, :10]:
        print(' %s' % terms[j], end='')
    print()
    

Want to Learn More About
Clustering sparse data with MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=True)
Init 1/3 with method: k-means++
Inertia for init 1/3: 197.546453
Init 2/3 with method: k-means++
Inertia for init 2/3: 223.546049
Init 3/3 with method: k-means++
Inertia for init 3/3: 220.169478
Minibatch iteration 1/400: mean batch inertia: 0.690426, ewa inertia: 0.690426 
Minibatch iteration 2/400: mean batch inertia: 0.777423, ewa inertia: 0.739438 
Minibatch iteration 3/400: mean batch inertia: 0.726256, ewa inertia: 0.732012 
Minibatch iteration 4/400: mean batch inertia: 0.758468, ewa inertia: 0.746917 
Minibatch iteration 5/400: mean batch inertia: 0.696617, ewa inertia: 0.718579 
Minibatch iteration 6/400: mean batch inertia: 0.676811, ewa inertia: 0.695047 
Minibatch iteration 7/400: mean 

In [9]:
X

<354x322 sparse matrix of type '<class 'numpy.float64'>'
	with 1887 stored elements in Compressed Sparse Row format>

In [28]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=500, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=None)